<a href="https://colab.research.google.com/github/VctrEman/Thermodynamics-Grupo-de-Desenvolvimento-Aeroespacial/blob/master/Classe_Biomassa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#classe para biomassa 3
class biomassa(object):
    
    #tabela de entalpias para produtos da exaustão, temperatura (C) - entalpia (kj/kg) 
    cts = \
       {100:{ 'ct_CO2': 169.98, 'ct_N2': 129.79, 'ct_H2O': 150.72, 'ct_ar': 132.3,'ct_A': 80.80 },
        200:{ 'ct_CO2': 357.55, 'ct_N2': 260, 'ct_H2O': 304.38, 'ct_ar': 266.28,'ct_A': 169.14 },
        300:{ 'ct_CO2': 558.93, 'ct_N2': 391.88, 'ct_H2O': 462.64, 'ct_ar': 402.77,'ct_A': 263.76 },
        400:{ 'ct_RO2' :0, 'ct_CO2': 772.04, 'ct_N2': 526.69, 'ct_H2O': 626.34, 'ct_ar': 542.61,'ct_A': 360 },}
   
   #atributos da biomassa
    def __init__(self, C, H, O, N, S = 0, W = 0, A = 0, Va = 0, lamb = 0, PCIt = 0, Vg = 0, Vgo = 0, Hg = 0, Mi = 0):
        self.C = C                                  
        self.H = H
        self.O = O
        self.N = N
        self.S = S
        self.A = A
        self.W = W
        self.Va = Va                            #ar teórico m3/kg
        self.lamb = lamb                        #lambda
        self.Var = 0                            #ar real m3/kg
        self.PCIt = PCIt                        
        self.Vgo = Vgo                          #volume teórico da exaustão m3/kg
        self.Vg = Vg                            #volume real m3/kg
        self.Mi = Mi                            #teor de cinzas g/m3
        self.Hg = Hg                            #entalpia kj/kg
    
    #metodo para exibir composicao da base seca
    #metodo para o ar teorico, real
    #metodo para o PCI bs
    #metodo para volume de gases da combustao a partir da composicao
    #metodo para vg da exaustao
    #metodo para Mi (cinzas)
    #metodo para hg

    def comp(self):
        print(f"""  
Composição percentual em base seca (%)\n
C:\t{round(self.C,2)}
H:\t{round(self.H,2)}
O:\t{round(self.O,2)}
N:\t{round(self.N,2)}
S:\t{round(self.S,2) }
A:\t{round(self.A,2)}
W:\t{round(self.W,2)}\n""")
    
    #metodo para mostrar os atributos físicos da biomassa
    def report(self, temp = 200, d = {} ):
        biomassa.Var_Teorico(self)
        biomassa.Mi(self)
        biomassa.Vg(self)
        l = ['C','O','H','N','S','A','W']
        for i in [item for item in d.items() if item[0] not in l]:
            print(f"{i[0]}:\t{round(i[1],2)}")

    #metodo para obtenção da biomassa em base seca
    def Bs(C, H, O, N, S, A, Wt, show = False):
        bu = [C,H,O,N,S,A,Wt] 
        bs =  [i - i*Wt/100 if i!= bu[-1] else  Wt  for i in bu] #composição do combustivel em base seca desconsiderando a agua
        if show:
            print(f'Composição percentual em base seca (%)\nC\t{bs[0]}\nH\t{bs[1]}\nO:\t{bs[2]}\nN:\t{bs[3]}\nS:\t{bs[4]}\nA:\t{bs[5]}')
        else: return dict(zip(['C','H','O','N','S','A','W'],bs))

    #calculo do ar teórico
    def Var_Teorico(self, **kwargs): 
        self.Va = (0.0888*(self.C + 0.375*self.S)+0.265*self.H - 0.0333*self.O)#  #m3 ar/kg comb
        biomassa.Vgo(self)
        biomassa.PCIt(self)
        self.Var = self.Va*self.lamb
        return self.Va
    
    #calculo do PCI em base trabalho/seca
    def PCIt(self): 
        self.PCIt = (339*self.C +1030*self.H -109*(self.O -self.S)-24*self.W)/1000   #kJ/kg #base seca
        return self.PCIt #kJ/kg #base seca

    #teor de cinzas
    def Mi(self): 
        self.Mi = self.A*10*0.6/biomassa.Vg(self)
        return self.Mi

    #entalpia dos gases de exautao
    def hg(self, ct_CO2=0, ct_N2=0, ct_H2O=0, ct_ar=0, ct_A=0 ,**kwags ):   
        hgo = biomassa.VRO2(self)*ct_CO2 + biomassa.VN2(self)*ct_N2 + biomassa.VH2O(self)*ct_H2O + self.A*0.6/100*ct_A #1/biomassa.Mi(self)/1000*ct_A  #self.A*0.6/100
        hao = (self.lamb - 1)*biomassa.Var_Teorico(self)*ct_ar
        self.Hg = hgo+hao
        return  self.Hg

    #volume dos gases de exaustão teórico, m3/kg
    def Vgo(self): 
        self.Vgo = biomassa.VN2(self) + biomassa.VH2O(self) +biomassa.VRO2(self)
        return self.Vgo

    #volume dos gases de exaustão, m3/kg
    def Vg(self): 
        self.Vg = biomassa.Vgo(self) + 1.0161*(self.lamb-1)*biomassa.Var_Teorico(self)
        return self.Vg

    #composição do volume dos componentes da exaustao
    def VN2(self, lamb = 1): return 0.79* (lamb) *self.Va + 0.008*self.N   
    def VRO2(self): return 0.01866*(self.C + 0.375*self.S)
    def VH2O(self): return 0.111*self.H + 0.01214*self.W + 0.0161*self.Va
    def VSO2(self): return 0.00699*self.S
    def VO2(self): return 0.21*(self.lamb -1)*self.Va
    def VCO2(self): return 0.01866*self.C

In [36]:
arroz1 = (41,4.3,35.9,0.4,0,18.3, 0)
arroz2 = (38.5, 5.7, 39.8, 0.5, 0, 15.5, 0 )
arroz3 = (39.4, 4.9, 37.1, 0.5, 0.1, 18, 0)
bagac = (44.8, 5.3, 39.6, 0.4, 0, 9.8, 0)
borra1 = (59.5, 7.3, 30.7, 2.5, 0, 0, 0) 
borra2 = (56.2, 7.1, 34.1, 2.4, 0.2, 0, 0) 
serra = (48.3, 5.9, 45.1, 0.2, 0, 0.4, 0) 

arrozm = [(i+j+k)/3 for i, j, k in zip(arroz1, arroz2, arroz3)]     #composição media da biomassa do arroz
borram = [(i+j)/2 for i, j in zip(borra1, borra2)]                  #composição media da biomassa da borra de café

In [37]:
#casca de arroz
p1 = biomassa(**biomassa.Bs(*arrozm) )
p1.W = 3.5  
p1.comp()
p1.Va = p1.Var_Teorico()
p1.lamb = 21/(21-5.6)
p1.Hg = p1.hg(**biomassa.cts[200])
p1.report(d = p1.__dict__)   

  
Composição percentual em base seca (%)

C:	39.63
H:	4.97
O:	37.6
N:	0.47
S:	0.03
A:	17.27
W:	3.5

Va:	3.58
lamb:	1.36
Var:	4.89
PCIt:	14.37
Vgo:	4.23
Vg:	5.55
Mi:	18.66
Hg:	1564.69


In [38]:
#borra de cafe
p2 = biomassa(**biomassa.Bs(*borram) )
p2.W = 2.5
p2.comp()
p2.Va = p2.Var_Teorico()
p2.lamb = 21/(21-7.4) #
p2.Hg = p2.hg(**biomassa.cts[200])
p2.report(d = p2.__dict__)

  
Composição percentual em base seca (%)

C:	57.85
H:	7.2
O:	32.4
N:	2.45
S:	0.1
A:	0.0
W:	2.5

Va:	5.97
lamb:	1.54
Var:	9.22
PCIt:	23.45
Vgo:	6.74
Vg:	10.04
Mi:	0.0
Hg:	2764.11


In [39]:
#serragem
p3 = biomassa(**biomassa.Bs(*serra) )
p3.W = 10
p3.comp()
p3.Va = p1.Var_Teorico()
p3.lamb = 21/(21-6.3)
p3.Hg = p3.hg(**biomassa.cts[200])
p3.report(d = p3.__dict__)

  
Composição percentual em base seca (%)

C:	48.3
H:	5.9
O:	45.1
N:	0.2
S:	0
A:	0.4
W:	10

Va:	4.35
lamb:	1.43
Var:	6.22
PCIt:	17.29
Vgo:	5.19
Vg:	7.08
Mi:	0.34
Hg:	1809.72


In [40]:
#3º lista questão 2
l3_q2 = biomassa( **biomassa.Bs(41, 4.3, 35.9, 0.5, 0, 18.3, 10)  )
l3_q2.Va = l3_q2.Var_Teorico()
l3_q2.lamb = 21/(21-5.3)
l3_q2.Hg = l3_q2.hg(**biomassa.cts[200])
l3_q2.comp()
l3_q2.report(d = l3_q2.__dict__)

  
Composição percentual em base seca (%)

C:	36.9
H:	3.87
O:	32.31
N:	0.45
S:	0.0
A:	16.47
W:	10

Va:	3.23
lamb:	1.34
Var:	4.32
PCIt:	12.73
Vgo:	3.84
Vg:	4.95
Mi:	19.96
Hg:	1400.07


In [41]:
#3º lista questão 1
l3_q1 = biomassa( **biomassa.Bs( 49, 5.87, 43.97, 0.3, 0.01, 0.72, 30) )
l3_q1.lamb = 21/(21-7.01)
l3_q1.Va = l3_q1.Var_Teorico()
l3_q1.Hg = l3_q1.hg(**biomassa.cts[200])
l3_q1.comp()
l3_q1.report(d = l3_q1.__dict__)

  
Composição percentual em base seca (%)

C:	34.3
H:	4.11
O:	30.78
N:	0.21
S:	0.01
A:	0.5
W:	30

Va:	3.11
lamb:	1.5
Var:	4.67
PCIt:	11.79
Vgo:	3.97
Vg:	5.55
Mi:	0.54
Hg:	1548.49


In [42]:
#3º lista questão 3 
l3_q3 = biomassa( **biomassa.Bs(49.44, 6.09, 40.57, 0, 0, 3.4, 52)  )
l3_q3.comp()
l3_q3.Va = l3_q3.Var_Teorico()
l3_q3.lamb = 21/(21-8.7)
l3_q3.Hg = l3_q3.hg(**biomassa.cts[200])
l3_q3.report(d = l3_q3.__dict__)

  
Composição percentual em base seca (%)

C:	23.73
H:	2.92
O:	19.47
N:	0.0
S:	0.0
A:	1.63
W:	52

Va:	2.23
lamb:	1.71
Var:	3.81
PCIt:	7.69
Vgo:	3.2
Vg:	4.8
Mi:	2.04
Hg:	1341.28
